# \[Honor Task] Hadoop Streaming assignment 4: Word Groups

Calculate statistics for groups of words which are equal up to permutations of letters. For example, `emit`, `item` and `time` are the same words up to a permutation of letters. Determine such groups of words and sum all their counts. Apply stop words filter. Filter out groups that consist of only one word.

**Output:** count of occurrences for the group of words, number of unique words in the group, comma-separated list of the words in the group in lexicographical order:

`sum <tab> group size <tab> word1,word2,...`

**Example:** assume ‘emit’ occurred 3 times, 'item' -- 2 times, 'time' -- 5 times; 3 + 2 + 5 = 10, group contains 3 words, so for this group result is:

`10 3 emit,item,time`

The result of the task is the output line with word `english`.

The result on the sample dataset:

`7823    eghilns 5   english,helsing,hesling,shengli,shingle`

### Solution description.

The MapReduce schema for this task:
* `map`: `(<article> <text>) -> [(<permutation> <word> <count1>),]`
* `combine`: `(<permutation> <word> <count1>) -> [(<permutation> <word> <count2>),]`
* `reduce`: `(<permutation> [(<word> <count2>)]) -> [(<total_count> <unique_words_count> [<word>,]),]`

### Step 1. Create the mapper.

In [ ]:
%%writefile mapper.py
"""
This is a map function: 
  map: (<article> <text>) -> [(<permutation> <word> <count1>),]

Calculates all occurrences of a word in an article.
"""

import sys
import re

from collections import Counter


def get_stop_words():
    """
    Reads a file with stop words and parses it to set.
    """
    words = set()
    
    with open('stop_words_en.txt', 'r', encoding='utf-8') as f:
        words = {w.strip().lower() for w in f}
    
    return words


stop_words = get_stop_words()

# Main block
for line in sys.stdin:
    try:
        article_id, text = line.strip().split('\t', 1)
    except ValueError as e:
        continue

    words = [w.lower() for w in re.split(r"\W*\s+\W*", text, flags=re.UNICODE) if w.lower not in stop_words]
    counter = Counter(words)
    for word, count in counter.items():
        print("".join(sorted(word)), word, count, sep="\t")

### Step 2. Create the combiner.

In [ ]:
%%writefile combiner.py
"""
This is a combiner function: 
  combine: (<permutation> <word> <count1>) -> [(<permutation> <word> <count2>),]
  
  Calculates all occurrences of a word in several articles.
"""

import sys


current_word = None
current_permutation = None
word_count = 0

# Main block
for line in sys.stdin:
    try:
        permutation, word, count = line.strip().split('\t', 2)
    except ValueError as e:
        continue

    if current_word != word:
        if current_word:
            print(current_permutation, current_word, word_count, sep="\t")

        word_count = 0
        current_word = word
        current_permutation = permutation

    word_count += int(count)

if current_word:
    print(current_permutation, current_word, word_count, sep="\t")

### Step 3. Create the reducer.

In [ ]:
%%writefile reducer.py
"""
This is a reducer function: 
  reduce: (<permutation> [(<word> <count2>)]) -> [(<total_count> <unique_words_count> [<word>,]),]
"""

import sys


current_permutation = None
words = set()
words_count = 0
    
# Main block
for line in sys.stdin:
    try:
        permutation, word, count = line.strip().split('\t', 2)
    except ValueError as e:
        continue

    if current_permutation != permutation:
        if current_permutation and len(words) > 1:
            print(words_count, current_permutation, len(words), ",".join(words), sep="\t")

        words_count = 0
        words = set()
        current_permutation = permutation

    words_count += int(count)
    words.add(word)

if current_permutation and len(words) > 1:
    print(words_count, len(words), ",".join(sorted(words)), sep="\t")

### Step 4. Run MapReduce jobs.

In [ ]:
%%bash

INPUT="/data/wiki/en_articles_part"
OUT_DIR="coursera_mr_task4"
NUM_REDUCERS=4

hdfs dfs -rm -r -skipTrash ${OUT_DIR}/count > /dev/null

# Count words groups
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Word Groups (Count)" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keycomparator.options="-k1 -k2" \
    -files mapper.py,reducer.py,combiner.py,/datasets/stop_words_en.txt \
    -mapper "python3 mapper.py" \
    -combiner "python3 combiner.py" \
    -reducer "python3 reducer.py" \
    -input ${INPUT} \
    -output ${OUT_DIR}/count > /dev/null

hdfs dfs -rm -r -skipTrash ${OUT_DIR}/reduce > /dev/null

# Sort counts
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Word Groups (Reduce)" \
    -D mapreduce.job.reduces=1 \
    -mapper "cat" \
    -reducer "cat" \
    -input ${OUT_DIR}/count \
    -output ${OUT_DIR}/reduce > /dev/null

# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR}/reduce/part-00000 | grep -P '(,|\t)english($|,)' | head -1